In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv


load_dotenv() 

True

In [2]:
# enhanced_graph_prompt = PromptTemplate.from_template(
#     """
# Create a dialogue graph for a {topic} conversation that will be used for training data generation. The graph must follow these requirements:

# 1. Dialogue Flow Requirements:
#    - Each assistant message (node) must be a precise question or statement that expects a specific type of response
#    - Each user message (edge) must logically and directly respond to the previous assistant message
#    - All paths must maintain clear context and natural conversation flow
#    - Avoid any ambiguous or overly generic responses

# 2. Graph Structure Requirements:
#    - Must contain at least 2 distinct cycles (return paths)
#    - Each cycle should allow users to:
#      * Return to previous choices for modification
#      * Restart specific parts of the conversation
#      * Change their mind about earlier decisions
#    - Include clear exit points from each major decision path
   
# 3. Core Path Types:
#    - Main success path (completing the intended task)
#    - Multiple modification paths (returning to change choices)
#    - Early exit paths (user decides to stop)
#    - Alternative success paths (achieving goal differently)

# Example of a good cycle structure:
# Assistant: "What size coffee would you like?"
# User: "Medium please"
# Assistant: "Would you like that hot or iced?"
# User: "Actually, can I change my size?"
# Assistant: "Of course! What size would you like instead?"

# Format:
# {{
#     "edges": [
#         {{
#             "source": "node_id",
#             "target": "node_id",
#             "utterances": ["User response text"]
#         }}
#     ],
#     "nodes": [
#         {{
#             "id": "node_id",
#             "label": "semantic_label",
#             "is_start": boolean,
#             "utterances": ["Assistant message text"]
#         }}
#     ]
# }}

# Requirements for node IDs:
# - Must be unique integers
# - Start node should have ID 1
# - IDs should increment sequentially

# Return ONLY the valid JSON without any additional text or explanations.
# """
# )

# graph_generator = CycleGraphGenerator()

In [3]:
# def validate_graph_cycle_requirement(
#     graph: BaseGraph,
#     min_cycles: int = 2
# ) -> Dict[str, Any]:
#     """
#     Проверяет граф на соответствие техническим требованиям
    
#     Args:
#         graph: BaseGraph для проверки
#         min_cycles: минимальное требуемое количество циклов
        
#     Returns:
#         Dict с результатами проверки:
#         {
#             "meets_requirements": bool,
#             "cycles": List[List[int]],
#             "cycles_count": int
#         }
#     """
#     print("\n🔍 Checking graph requirements...")
    
#     try:
#         cycles = list(nx.simple_cycles(graph.graph))
#         cycles_count = len(cycles)
        
#         print(f"🔄 Found {cycles_count} cycles in the graph:")
#         for i, cycle in enumerate(cycles, 1):
#             print(f"Cycle {i}: {' -> '.join(map(str, cycle + [cycle[0]]))}")
            
#         meets_requirements = cycles_count >= min_cycles
        
#         if not meets_requirements:
#             print(f"❌ Graph doesn't meet cycle requirements (minimum {min_cycles} cycles needed)")
#         else:
#             print("✅ Graph meets cycle requirements")
            
#         return {
#             "meets_requirements": meets_requirements,
#             "cycles": cycles,
#             "cycles_count": cycles_count
#         }
        
#     except Exception as e:
#         print(f"❌ Validation error: {str(e)}")
#         raise

In [4]:
# from chatsky_llm_autoconfig.metrics.llm_metrics import graph_validation
# import json
# repair_template = PromptTemplate.from_template("""
# Fix the invalid transitions in this dialogue graph while keeping its structure.

# Current invalid transitions that need to be fixed:
# {invalid_transitions}

# Original graph structure:
# {graph_json}

# Requirements for the fix:
# 1. Keep all node IDs and structure the same
# 2. Fix ONLY the invalid transitions
# 3. Make sure the fixed transitions are logical and natural
# 4. Each user response must logically follow from the assistant's previous message
# 5. Each assistant response must properly address the user's input

# Return ONLY the complete fixed graph JSON with the same structure.
# """)

# def check_and_fix_transitions(graph: BaseGraph, graph_generator: CycleGraphGenerator, model: BaseChatModel, max_attempts: int = 3) -> Dict[str, Any]:
#     """
#     Проверяет переходы в графе и пытается исправить невалидные через LLM
    
#     Args:
#         graph: Исходный граф для проверки и исправления
#         graph_generator: Генератор графов для исправления
#         model: Модель для валидации
#         max_attempts: Максимальное количество попыток исправления
        
#     Returns:
#         Dict: {
#             "is_valid": bool,  # Удалось ли получить валидный граф
#             "graph": BaseGraph,  # Последняя версия графа (исправленная или нет)
#             "validation_details": {  # Детали последней валидации
#                 "invalid_transitions": [...],  # Список оставшихся невалидных переходов
#                 "attempts_made": int,  # Сколько попыток исправления было сделано
#                 "fixed_count": int,  # Сколько переходов удалось исправить
#             }
#         }
#     """
#     # Проверяем изначальный граф
#     initial_validation = graph_validation(graph, model)
#     if initial_validation["is_valid"]:
#         return {
#             "is_valid": True,
#             "graph": graph,
#             "validation_details": {
#                 "invalid_transitions": [],
#                 "attempts_made": 0,
#                 "fixed_count": 0
#             }
#         }
    
#     initial_invalid_count = len(initial_validation["invalid_transitions"])
#     current_graph = graph
#     current_attempt = 0
    
#     while current_attempt < max_attempts:
#         print(f"\n🔄 Fix attempt {current_attempt + 1}/{max_attempts}")
        
           
#         try:
#             # Используем graph_generator для генерации исправленного графа
#             current_graph = graph_generator.invoke(model=model, prompt=repair_template, invalid_transitions=initial_validation["invalid_transitions"], graph_json=current_graph.graph_dict)
            
#             # Проверяем исправленный граф
#             validation = graph_validation(current_graph, model)
#             if validation["is_valid"]:
#                 return {
#                     "is_valid": True,
#                     "graph": current_graph,
#                     "validation_details": {
#                         "invalid_transitions": [],
#                         "attempts_made": current_attempt + 1,
#                         "fixed_count": initial_invalid_count
#                     }
#                 }
                
#         except Exception as e:
#             print(f"⚠️ Error during fix attempt: {str(e)}")
#             break
            
#         current_attempt += 1
    
#     remaining_invalid = len(validation["invalid_transitions"])
    
#     return {
#         "is_valid": False,
#         "graph": current_graph,
#         "validation_details": {
#             "invalid_transitions": validation["invalid_transitions"],
#             "attempts_made": current_attempt,
#             "fixed_count": initial_invalid_count - remaining_invalid
#         }
#     }

In [5]:
# from chatsky_llm_autoconfig.algorithms.dialogue_generation import  RecursiveDialogueSampler
# from chatsky_llm_autoconfig.metrics.automatic_metrics import all_utterances_present
# from chatsky_llm_autoconfig.metrics.llm_metrics import graph_validation, is_theme_valid

# CYCLE_REQUIREMENT = 2

#  # Генерация
# gen_model = ChatOpenAI(
#     model='o1-mini',
#     api_key=os.getenv("OPENAI_API_KEY"),
#     base_url=os.getenv("OPENAI_BASE_URL"),
#     temperature=1
# )

# try:
#     topic = "ordering pizza"
        
#     graph = graph_generator.invoke(model=gen_model, prompt=enhanced_graph_prompt, topic=topic)

#     # Проверка требований
#     validation_result = validate_graph_cycle_requirement(graph, min_cycles=CYCLE_REQUIREMENT)

#     # Семплинг диалогов
#     dial_sampler = RecursiveDialogueSampler()
#     sampled_dialogues = dial_sampler.invoke(graph, 1, -1)

#     # Проверка семплинга
#     sampling_result = all_utterances_present(graph, sampled_dialogues)
    
#     if sampling_result is False:
#         raise ValueError("Failed to sample valid dialogues from the graph or sampling error occurred")
        
#     # Сначала проверяем валидность темы
#     theme_validation = is_theme_valid(graph, gen_model, topic)
#     if not theme_validation['value']:
#         raise ValueError(f"Theme validation failed: {theme_validation['description']}")
    
#     # Если тема валидна, проверяем триплеты в цикле
#     transition_validation = check_and_fix_transitions(graph, graph_generator, gen_model, max_attempts=3)
    
#     print("\nValidation results:")
#     print(f"Theme valid: {theme_validation['value']}")
#     print(f"Transitions valid: {transition_validation['is_valid']}")
    
#     if not transition_validation['is_valid']:
#         print("\nInvalid transitions:")
#         for t in transition_validation['invalid_transitions']:
#             print(f"- {t['reason']}")

# except Exception as e:
#     print(f"❌ Error during graph generation or validation: {str(e)}")
#     raise


In [ ]:
from chatsky_llm_autoconfig.algorithms.cycle_graph_generation_pipeline import GraphGenerationPipeline
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from chatsky_llm_autoconfig.schemas import GraphGenerationResult
from datetime import datetime
from pathlib import Path
import os
import json


def generate_graphs():
    output_dir = Path("generated_graphs")
    output_dir.mkdir(exist_ok=True)
    
    generation_model = ChatOpenAI(
        model='o1-mini',
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_BASE_URL"),
        temperature=1
    )
    
    validation_model = ChatOpenAI(
        model='gpt-4o',
        api_key=os.getenv("OPENAI_API_KEY"),
        base_url=os.getenv("OPENAI_BASE_URL"),
        temperature=0
    )
    
    pipeline = GraphGenerationPipeline(
        generation_model=generation_model,
        validation_model=validation_model
    )
    
    topics = [
        "technical support conversation",
        # "restaurant reservation",
        # "online shopping checkout",
        # "job interview",
        # "travel booking"
    ]
    
    successful_generations = []
    
    for topic in topics:
        print(f"\n{'='*50}")
        print(f"Generating graph for topic: {topic}")
        print(f"{'='*50}")
        
        try:
            result = pipeline(topic)
            
            # Проверяем тип результата
            if isinstance(result, GraphGenerationResult):
                print(f"✅ Successfully generated graph for {topic}")
                # Сохраняем полный результат с графом и диалогами
                successful_generations.append({
                    "graph": result.graph,
                    "topic": result.topic,
                    "dialogues": result.dialogues
                })
            else:  # isinstance(result, GenerationError)
                print(f"❌ Failed to generate graph for {topic}")
                print(f"Error type: {result.error_type}")
                print(f"Error message: {result.message}")
                    
        except Exception as e:
            print(f"❌ Unexpected error processing topic '{topic}': {str(e)}")
            continue
    
    if successful_generations:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"generated_graphs_{timestamp}.json"
        with open(output_dir / filename, "w", encoding="utf-8") as f:
            # Используем model_dump() для корректной сериализации Pydantic моделей
            json_data = [result for result in successful_generations]
            json.dump(json_data, f, indent=2, ensure_ascii=False)
            print(f"\nAll graphs saved to: {filename}")
        
        print(f"\nSuccessfully generated {len(successful_generations)} graphs out of {len(topics)} topics")
    else:
        print("\nNo graphs were successfully generated")


if __name__ == "__main__":
    generate_graphs()

/Users/dmitriimartynov/Documents/Projects/chatsky-llm-autoconfig/.venv/lib/python3.12/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "validate" in "Dialogue" shadows an attribute in parent "BaseModel"
  warnings.warn(



Generating graph for topic: technical support conversation
Generating Graph ...


INFO:httpx:HTTP Request: POST http://193.187.173.33:8002/api/providers/openai/v1/chat/completions "HTTP/1.1 200 OK"



🔍 Checking graph requirements...
🔄 Found 30 cycles in the graph:
Cycle 1: 1 -> 2 -> 5 -> 7 -> 11 -> 1
Cycle 2: 1 -> 2 -> 5 -> 7 -> 12 -> 1
Cycle 3: 1 -> 2 -> 5 -> 8 -> 11 -> 1
Cycle 4: 1 -> 2 -> 5 -> 8 -> 12 -> 1
Cycle 5: 1 -> 2 -> 6 -> 9 -> 11 -> 1
Cycle 6: 1 -> 2 -> 6 -> 9 -> 12 -> 1
Cycle 7: 1 -> 2 -> 6 -> 10 -> 11 -> 1
Cycle 8: 1 -> 2 -> 6 -> 10 -> 12 -> 1
Cycle 9: 1 -> 2 -> 1
Cycle 10: 1 -> 3 -> 14 -> 16 -> 1
Cycle 11: 1 -> 3 -> 14 -> 17 -> 11 -> 1
Cycle 12: 1 -> 3 -> 14 -> 17 -> 16 -> 1
Cycle 13: 1 -> 3 -> 15 -> 18 -> 1
Cycle 14: 1 -> 3 -> 15 -> 19 -> 11 -> 1
Cycle 15: 1 -> 3 -> 15 -> 19 -> 18 -> 1
Cycle 16: 1 -> 3 -> 1
Cycle 17: 1 -> 4 -> 20 -> 1
Cycle 18: 1 -> 4 -> 1
Cycle 19: 1 -> 4 -> 21 -> 20 -> 1
Cycle 20: 1 -> 4 -> 21 -> 1
Cycle 21: 3 -> 14 -> 17 -> 3
Cycle 22: 3 -> 14 -> 3
Cycle 23: 3 -> 15 -> 19 -> 3
Cycle 24: 3 -> 15 -> 3
Cycle 25: 2 -> 5 -> 7 -> 2
Cycle 26: 2 -> 5 -> 8 -> 2
Cycle 27: 2 -> 5 -> 2
Cycle 28: 2 -> 6 -> 9 -> 2
Cycle 29: 2 -> 6 -> 10 -> 2
Cycle 30: 2 -> 6 -